## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Fetch id list

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list


In [ ]:
task_name = "AllTask"
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]

saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
    conn,
    task_name=None, #['FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly'],
    dose_mult=2,
    genotype="white",
    bad_ids=bad_id,
    csv_prefix="dlc_table",   # -> dlc_table_saline.csv, dlc_table_ghrelin.csv, ...
    min_trial_length=None     # or 600
)

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"exc_id: {exc_id}\n")
print(f"inh_id: {inh_id}\n")


## Calculate curvature

In [ ]:
import importlib
import Python_scripts.Feature_functions.trajectory_curvature
import Python_scripts.Data_analysis.plot_groupwise_bar
import Python_scripts.Data_analysis.compare_distributions

importlib.reload(Python_scripts.Feature_functions.trajectory_curvature)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)
importlib.reload(Python_scripts.Data_analysis.compare_distributions)

from Python_scripts.Feature_functions.trajectory_curvature import (
    compute_trajectory_curvature, batch_trajectory_curvature
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar
from Python_scripts.Data_analysis.compare_distributions import compare_distributions


## Batch call with different input parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- Configure once ---
bodypart = 'Midback'
ycol = 'mean_curvature'
window_range = range(11, 28, 4)

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":    saline_id,   # e.g., list/array of trial IDs
    "Ghrelin":   ghrelin_id,
    # "Inhibitory": inh_id,
    # "Excitatory": exc_id,
}

# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_10X_{task_name}_curvature_window_sweep.pdf"
with PdfPages(outfile) as pdf:
    for window_size in window_range:
        print(f"[INFO] Analyzing window_size = {window_size}")

        # Compute all groups for this window size
        frames = []
        for label, ids in group_specs.items():
            df = batch_trajectory_curvature(
                conn, ids, bodypart=bodypart, window=window_size
            )
            df["group"] = label
            frames.append(df)

        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue

        df_curv = pd.concat(frames, ignore_index=True)

        # Plot
        order = list(group_specs.keys())  # consistent ordering across pages
        fig, ax = plot_groupwise_bar(
            df_curv, y='mean_curvature',
            ylabel='Mean curvature',
            plot_type='bar', show_points=False,
            order=order, show_stats=True,
            tests_to_show=("ranksums","ttest", "anova")  # optional
        )

        ax.set_title(f"{task_name} | window size = {window_size:d}", pad=20)

        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")


## Single call with fixed hyper parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- Configure once ---
bodypart = 'Midback'
ycol = 'mean_curvature'
window_size = 23

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":    saline_id,   # e.g., list/array of trial IDs
    # "Ghrelin":   ghrelin_id,
    "Inhibitory": inh_id,
    "Excitatory": exc_id,
}

# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

# Compute all groups for this window size
frames = []
for label, ids in group_specs.items():
    df = batch_trajectory_curvature(
        conn, ids, bodypart=bodypart, window=window_size
    )
    df["group"] = label
    frames.append(df)

if not frames:
    print("[WARN] No groups provided—skipping this window.")
    sys.exit(0)

df_curv = pd.concat(frames, ignore_index=True)

# Plot
order = list(group_specs.keys())  # consistent ordering across pages
fig, ax = plot_groupwise_bar(
    df_curv, y='mean_curvature',
    ylabel='Mean curvature',
    plot_type='bar', show_points=False,
    order=order, show_stats=True,
    tests_to_show=("ranksums","ttest", "anova")  # optional
)

ax.set_title(f"{task_name} | window size = {window_size:d}", pad=20)

fig.savefig(f"White_{task_name}_modulation_curvature.pdf", dpi=300, bbox_inches='tight')
